In [1]:
# import necessary modules
import pandas as pd
import os

In [2]:
# define the source data paths
students_csvPath = os.path.join('Resources', 'students_complete.csv')
schools_csvPath = os.path.join('Resources', 'schools_complete.csv')

In [3]:
# read the data into their initial dataframs
source_students_df = pd.read_csv(students_csvPath)
source_schools_df = pd.read_csv(schools_csvPath)

In [4]:
# rename columns to use universal formatting
source_students_df = source_students_df.rename(columns={
                                    'Student ID': 'Student_ID',
                                    'student_name': 'Student_Name',
                                    'gender': 'Gender',
                                    'grade': 'Grade',
                                    'school_name': 'School_Name',
                                    'reading_score': 'Reading_Score',
                                    'math_score': 'Math_Score'})

source_schools_df = source_schools_df.rename(columns={
                                    'School ID': 'School_ID',
                                    'school_name': 'School_Name',
                                    'type': 'School_Type',
                                    'size': 'Total_Students',
                                    'budget': 'Total_School_Budget'})

# add a counting column to the source_students_df dataframe
source_students_df['Student_Counter'] = 1

In [5]:
# define threshold values
passingThreshold = 70

In [6]:
# calculate the district summary metrics
dsum_totalSchools = source_schools_df.nunique()['School_Name']
dsum_totalStudents = source_schools_df['Total_Students'].sum()
dsum_totalBudget = source_schools_df['Total_School_Budget'].sum()
dsum_avgMathScore = source_students_df['Math_Score'].mean()
dsum_avgReadingScore = source_students_df['Reading_Score'].mean()
dsum_passingMath = sum(source_students_df.Math_Score >= passingThreshold) / dsum_totalStudents * 100
dsum_passinReading = sum(source_students_df.Reading_Score >= passingThreshold) / dsum_totalStudents * 100
dsum_passingBoth = sum(
                (source_students_df.Math_Score >= passingThreshold) &
                (source_students_df.Reading_Score >= passingThreshold)) / dsum_totalStudents * 100

In [7]:
# create and format the district summary DataFrame
district_summary_df = pd.DataFrame({
    'Total Schools': [dsum_totalSchools],
    'Total Students': f'{dsum_totalStudents:,.0f}',
    'Total Budget': f'${dsum_totalBudget:,.2f}',
    'Avg Math Score': f'{dsum_avgMathScore:,.2f}',
    'Avg Reading Score': f'{dsum_avgReadingScore:,.2f}',
    'Passing Math': f'{dsum_passingMath:,.2f}%',
    'Passing Reading': f'{dsum_passinReading:,.2f}%',
    'Passing': f'{dsum_passingBoth:,.2f}%'})

# display the 
district_summary_df.style.hide_index()

Total Schools,Total Students,Total Budget,Avg Math Score,Avg Reading Score,Passing Math,Passing Reading,Passing
15,"39,170","$24,649,428.00",78.99,81.88,74.98%,85.81%,65.17%


In [8]:
# create filtered dataframes that meet various criteria
passingMath_df = source_students_df.loc[source_students_df['Math_Score'] >= passingThreshold, :]
passingRead_df = source_students_df.loc[source_students_df['Reading_Score'] >= passingThreshold, :]
passingBoth_df = source_students_df.loc[
        (source_students_df['Math_Score'] >= passingThreshold) & 
        (source_students_df['Reading_Score'] >= passingThreshold)]

In [9]:
# merge the relevant datasets
schools_summary_df = pd.merge(source_schools_df, source_students_df.groupby('School_Name').sum(), on='School_Name')
math_merged_df = pd.merge(source_schools_df, passingMath_df.groupby('School_Name').count(), on='School_Name')
read_merged_df = pd.merge(source_schools_df, passingRead_df.groupby('School_Name').count(), on='School_Name')
both_merged_df = pd.merge(source_schools_df, passingBoth_df.groupby('School_Name').count(), on='School_Name')

In [10]:
# create the summary school dataframe
schools_summary_df['Per_Student_Budget'] = schools_summary_df['Total_School_Budget'] / schools_summary_df['Total_Students']
schools_summary_df['Average_Math_Score'] = schools_summary_df['Math_Score'] / schools_summary_df['Total_Students']
schools_summary_df['Average_Reading_Score'] = schools_summary_df['Reading_Score'] / schools_summary_df['Total_Students']
schools_summary_df['Passing_Math'] = math_merged_df['Student_Counter'] / math_merged_df['Total_Students'] * 100
schools_summary_df['Passing_Reading'] = read_merged_df['Student_Counter'] / read_merged_df['Total_Students'] * 100
schools_summary_df['Passing_Both'] = both_merged_df['Student_Counter'] / both_merged_df['Total_Students'] * 100

In [11]:
# reorder the columns
reordered_schools_summary_df = schools_summary_df[[
                                'School_Name',
                                'School_Type',
                                'Total_Students',
                                'Total_School_Budget',
                                'Per_Student_Budget',
                                'Average_Math_Score',
                                'Average_Reading_Score',
                                'Passing_Math',
                                'Passing_Reading',
                                'Passing_Both']]

In [12]:
# rename the columns
renamed_schools_summary_df = reordered_schools_summary_df.rename(columns={
                                'School_Name': 'School Name',
                                'School_Type': 'School Type',
                                'Total_Students': 'Total Students',
                                'Total_School_Budget': 'Total School Budget',
                                'Per_Student_Budget': 'Per Student Budget',
                                'Average_Math_Score': 'Average Math Score',
                                'Average_Reading_Score': 'Average Reading Score',
                                'Passing_Math': '% Passing Math',
                                'Passing_Reading': '% Passing Reading',
                                'Passing_Both': '% Overall Passing'})

In [13]:
# create a copy for column reformatting
reformatted_schools_summary_df = renamed_schools_summary_df.copy(deep=False)

# reformat the columns
reformatted_schools_summary_df['Total Students'] = reformatted_schools_summary_df['Total Students'].map('{:,.0f}'.format)
reformatted_schools_summary_df['Total School Budget'] = reformatted_schools_summary_df['Total School Budget'].map('${:,.2f}'.format)
reformatted_schools_summary_df['Per Student Budget'] = reformatted_schools_summary_df['Per Student Budget'].map('${:,.2f}'.format)
reformatted_schools_summary_df['Average Math Score'] = reformatted_schools_summary_df['Average Math Score'].map('{:,.2f}'.format)
reformatted_schools_summary_df['Average Reading Score'] = reformatted_schools_summary_df['Average Reading Score'].map('{:,.2f}'.format)
reformatted_schools_summary_df['% Passing Math'] = reformatted_schools_summary_df['% Passing Math'].map('{:,.2f}%'.format)
reformatted_schools_summary_df['% Passing Reading'] = reformatted_schools_summary_df['% Passing Reading'].map('{:,.2f}%'.format)
reformatted_schools_summary_df['% Overall Passing'] = reformatted_schools_summary_df['% Overall Passing'].map('{:,.2f}%'.format)

In [14]:
# sort the reformatted dataframe
final_school_summary_df = reformatted_schools_summary_df.sort_values('School Name', ascending=True).reset_index(drop=True)
final_school_summary_df.style.hide_index()

School Name,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Bailey High School,District,"4,976","$3,124,928.00",$628.00,77.05,81.03,66.68%,81.93%,54.64%
Cabrera High School,Charter,"1,858","$1,081,356.00",$582.00,83.06,83.98,94.13%,97.04%,91.33%
Figueroa High School,District,"2,949","$1,884,411.00",$639.00,76.71,81.16,65.99%,80.74%,53.20%
Ford High School,District,"2,739","$1,763,916.00",$644.00,77.10,80.75,68.31%,79.30%,54.29%
Griffin High School,Charter,"1,468","$917,500.00",$625.00,83.35,83.82,93.39%,97.14%,90.60%
Hernandez High School,District,"4,635","$3,022,020.00",$652.00,77.29,80.93,66.75%,80.86%,53.53%
Holden High School,Charter,427,"$248,087.00",$581.00,83.80,83.81,92.51%,96.25%,89.23%
Huang High School,District,"2,917","$1,910,635.00",$655.00,76.63,81.18,65.68%,81.32%,53.51%
Johnson High School,District,"4,761","$3,094,650.00",$650.00,77.07,80.97,66.06%,81.22%,53.54%
Pena High School,Charter,962,"$585,858.00",$609.00,83.84,84.04,94.59%,95.95%,90.54%


In [20]:
# display the top five schools by % Overall Passing
best_schools_df = final_school_summary_df.sort_values('% Overall Passing', ascending=False)
best_schools_df.head(5).style.hide_index()

School Name,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Cabrera High School,Charter,"1,858","$1,081,356.00",$582.00,83.06,83.98,94.13%,97.04%,91.33%
Thomas High School,Charter,"1,635","$1,043,130.00",$638.00,83.42,83.85,93.27%,97.31%,90.95%
Griffin High School,Charter,"1,468","$917,500.00",$625.00,83.35,83.82,93.39%,97.14%,90.60%
Wilson High School,Charter,"2,283","$1,319,574.00",$578.00,83.27,83.99,93.87%,96.54%,90.58%
Pena High School,Charter,962,"$585,858.00",$609.00,83.84,84.04,94.59%,95.95%,90.54%


In [23]:
# display the bottom five schools by % Overall Passing
worst_schools_df = final_school_summary_df.sort_values('% Overall Passing', ascending=True)
worst_schools_df.head(5).style.hide_index()

School Name,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Rodriguez High School,District,"3,999","$2,547,363.00",$637.00,76.84,80.74,66.37%,80.22%,52.99%
Figueroa High School,District,"2,949","$1,884,411.00",$639.00,76.71,81.16,65.99%,80.74%,53.20%
Huang High School,District,"2,917","$1,910,635.00",$655.00,76.63,81.18,65.68%,81.32%,53.51%
Hernandez High School,District,"4,635","$3,022,020.00",$652.00,77.29,80.93,66.75%,80.86%,53.53%
Johnson High School,District,"4,761","$3,094,650.00",$650.00,77.07,80.97,66.06%,81.22%,53.54%
